In [77]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
from keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.text import one_hot

In [68]:
train= pd.read_csv(r'C:\Users\vikil\Desktop\ML\data_Sets\twitter\train.csv')
test=pd.read_csv(r'C:\Users\vikil\Desktop\ML\data_Sets\twitter\test.csv')

In [69]:
train=train.drop(['keyword','location'],axis=1)
test=test.drop(['keyword','location'],axis=1)

In [70]:
ps = PorterStemmer()

In [71]:
corpus=[]
for i in range(0,len(train)):
    review=re.sub('[^a-z]',' ',train['text'][i])
    review=review.lower()
    review=review.split()
    
    review= [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [73]:
voc_size=5000

In [82]:
onehot_repr=[one_hot(words, voc_size) for words in corpus]

In [83]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [86]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [109]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [89]:
x_final=np.array(embedded_docs)

In [91]:
y_final=np.array(train['target'])

In [110]:
model.fit(x_final,y_final,epochs=10,batch_size=64)

Epoch 1/10
7613/7613 [==============================] - 26s 3ms/sample - loss: 0.5798 - acc: 0.6903
Epoch 2/10
7613/7613 [==============================] - 21s 3ms/sample - loss: 0.3900 - acc: 0.8316
Epoch 3/10
7613/7613 [==============================] - 22s 3ms/sample - loss: 0.3153 - acc: 0.8736
Epoch 4/10
7613/7613 [==============================] - 21s 3ms/sample - loss: 0.2594 - acc: 0.9027
Epoch 5/10
7613/7613 [==============================] - 21s 3ms/sample - loss: 0.2068 - acc: 0.9257
Epoch 6/10
7613/7613 [==============================] - 21s 3ms/sample - loss: 0.1617 - acc: 0.9480
Epoch 7/10
7613/7613 [==============================] - 22s 3ms/sample - loss: 0.1350 - acc: 0.9573
Epoch 8/10
7613/7613 [==============================] - 24s 3ms/sample - loss: 0.1173 - acc: 0.9628
Epoch 9/10
7613/7613 [==============================] - 24s 3ms/sample - loss: 0.1009 - acc: 0.9695
Epoch 10/10
7613/7613 [==============================] - 24s 3ms/sample - loss: 0.0834 - acc: 0.9746

In [97]:
test

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Storm in RI worse than last hurricane. My city...
3260,10868,Green Line derailment in Chicago http://t.co/U...
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...


In [99]:
corpus_test=[]
for i in range(0,len(test)):
    review=re.sub('[^a-z]',' ',test['text'][i])
    review=review.lower()
    review=review.split()
    
    review= [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus_test.append(review)

In [101]:
onehot_repr_test=[one_hot(words, voc_size) for words in corpus_test]

In [102]:
embedded_docs_test=pad_sequences(onehot_repr_test,padding='pre',maxlen=sent_length)

In [103]:
test_data=np.array(embedded_docs_test)

In [111]:
final_pred=model.predict_classes(test_data)

In [107]:
test_id = test.id

In [112]:
sub = pd.DataFrame()
sub['Id'] = test_id
sub['target'] = final_pred
sub.to_csv(r'C:\Users\vikil\Desktop\ML\data_Sets\twitter\submission_new_seq.csv',index=False)